<a href="https://colab.research.google.com/github/ssnirgudkar/Home-AIML/blob/main/IR_autolabel_5classes_batch1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install --upgrade segments-ai


In [ ]:
from segments import SegmentsClient
api_key = "a89182567b17766b91773021b18d04574cd75109"
client = SegmentsClient(api_key)

In [ ]:
dataset_identifier = "ssnirgudkar/PilotIR"


In [ ]:
#commenting this for now, since this will just give all the data sets available in my login
#dataset = client.get_dataset(dataset_identifier)
#print(dataset)


In [ ]:
from segments import SegmentsDataset
from segments.utils import export_dataset

# Initialize a SegmentsDataset from the release file. We will train using these 100 images (although they are 7 classes)
release_file = client.get_release(dataset_identifier,'V2.0')
#print(release_file)
releasedataset = SegmentsDataset(release_file, labelset='segmentation', filter_by='labeled')


In [ ]:
#install dependencies for detectron 2
!pip install pyyaml==5.1
import torch, torchvision
print(torch.__version__,torch.cuda.is_available())
!gcc --version



In [ ]:
!pip install torch==1.8.1

In [ ]:
#install detectron2 (Colab has CUDA 10.1 + torch 1.8)
import torch 
assert torch.__version__.startswith("1.8") #need to manually install 1.8 only if colab changes it's default version. as we see above, the default is 1.8 rt now
!pip install detectron2 -f https://dl.fbaipublicfiles.com/detectron2/wheels/cu101/torch1.8/index.html
# exit(0)  # After installation, you need to "restart runtime" in Colab. This line can also restart runtime



In [ ]:
#setup logger for detectron2 
import detectron2 
from detectron2.utils.logger import setup_logger 
setup_logger()

<Logger detectron2 (DEBUG)>

In [ ]:

#clone segments.ai repo for their utils code 
!git clone https://github.com/segments-ai/fast-labeling-workflow/
%cd fast-labeling-workflow


In [ ]:
from segments import SegmentsDataset
from segments.utils import export_dataset

# Initialize a SegmentsDataset from the release file
release_file = client.get_release(dataset_identifier,'V2.0')
#print(release_file)
releasedataset = SegmentsDataset(release_file, labelset='segmentation', filter_by='labeled')
from utils import train_model
model = train_model(releasedataset)

In [ ]:
#should save the model somewhere so it doesn't have to be recreated eachtime. this did not work
#model.save('/content/drive/MyDrive/Models/detectronwithir')


In [ ]:
#dataset_identifier_infer = "ssnirgudkar/IR-autolabel-5classes-batch1"
#dataset_infer = client.get_dataset(dataset_identifier_infer)

dataset_identifier_trial = "ssnirgudkar/trial"
dataset_infer_trial = client.get_dataset(dataset_identifier_trial)



In [ ]:
#infer_release_file = client.get_release(dataset_identifier_infer,'V1.1')
#infer_releasedataset = SegmentsDataset(infer_release_file, labelset='ground-truth', filter_by=['Unlabeled','Skipped','Prelabeled'])


trial_release_file = client.get_release(dataset_identifier_trial,'V1.1')
#trial_releasedataset = SegmentsDataset(trial_release_file, labelset='ground-truth', filter_by=['Unlabeled','Skipped','Prelabeled'])
trial_releasedataset = SegmentsDataset(trial_release_file, labelset='ground-truth', filter_by=['Prelabeled'])

In [ ]:
from segments import SegmentsDataset
from segments.utils import export_dataset
from segments.utils import bitmap2file

import cv2 
import numpy as np 
import os 
from utils import visualize 
#this needs 3 channel image. so had to convert grayscale IR to RGB - 3 channels by copying the same pixel value on all 3
#add logic to check if the image is 1 channel, and if so convert it to 3 channel
#for sample in infer_releasedataset:
for sample in trial_releasedataset:
   infer_image = sample['image'] 
      
   #imgcv = cv2.imread(infer_image)
   #assert not isinstance(imgcv,type(None)), 'image not found'
   #print ("file read = {0}".format(imgcv))
   #print(os.getcwd())  # prints current working directory
   #infer_image2 = RemoveAlphaChannel[infer_image, White]
   #4channel CYMK image is not taken by detectron. it needs 3 channels. so converting to RGB. we can add a conditional check
   # to first check if it's 4 channels and then convert only if it is
   infer_image2 = infer_image.convert("RGB")
   segmentation_bitmap, annotations = model(infer_image2)
   print("segmentation bitmap executed")
   #visualize the predictions 
   #visualize(infer_image2, segmentation_bitmap)
   #print(annotations)

# Upload the predictions to Segments.ai so that we can check the effectiveness of auto labels
   file = bitmap2file(segmentation_bitmap)
   asset = client.upload_asset(file, 'label.png')    
   attributes = {
        'format_version': '0.1',
        'annotations': annotations,
        'segmentation_bitmap': { 'url': asset['url'] },
   }
   client.add_label(sample['uuid'], 'ground-truth', attributes, label_status='PRELABELED')
   
  

segmentation bitmap executed


In [ ]:
# trying to print a picture from segments.io 
#client.get_samples(dataset_identifier_infer, per_page=1000, page=1)



#sample = client.get_sample(uuid="35a2c87c-69d1-4403-b413-565f114c3bd3")
#print(sample)



#infilename = "https://segmentsai-prod.s3.eu-west-2.amazonaws.com/assets/ssnirgudkar/eed2b567-b42f-4576-bb14-fcc547f9a24c.png" 
#img = cv2.imread(infilename, cv2.IMREAD_UNCHANGED)
#          assert not isinstance(img,type(None)), 'image not found'
#print ("file read = {0}".format(infilename))
          
#          img2 = img.copy()
#print(img[162,:])
#print(img)
#print(img.dtype)
#height,width = img.shape
#print (img.shape)

In [ ]:
# trial for 1 image from the set of 100 since auto labeling is not working
# adding sample image from amazon s3 to segments.ai dataset - snirgudkar / trial
# gives access denied error
from segments import SegmentsClient
api_key = "a89182567b17766b91773021b18d04574cd75109"
client = SegmentsClient(api_key)

dataset_identifier_trial = "ssnirgudkar/trial" 
name = '3607501.png' 
attributes = { 
              "image":{
                  #"url":"https://s3-us-east-2.amazonaws.com/ml-label-class3/1602782781.360750.png"
                  "url": "s3://ml-label-class3/1602782781.360750.png"
              }
}

client.add_sample(dataset_identifier_trial, name,attributes)